In [227]:
import pandas as pd
import numpy as np
from IPython.display import display
import openpyxl
from datetime import datetime
import requests

In [228]:
# You need coin recon sheet plus the APY/COFA sheet for live waterfall
# I download the sheets and save them as excel files so I have a copy of the raw data (for debugging and archiving)

In [229]:
def melt(df, col_vals, key, value):
    # melt pandas dataframe
    # col_vars: list of the columns that will be melted
    # key: name of column needs to be generated
    # value: name of the column that contains the value of interest
    keep_vars = df.columns.difference(col_vals)
    melted = []
    for c in col_vals:
        melted_c = df[keep_vars].copy()
        melted_c[key] = c
        melted_c[value] = df[c]
        melted.append(melted_c)
    return pd.concat(melted)

In [230]:
# CHECK IF THE FREEZE IS UPDATED
waterfall_freeze = pd.read_excel("excel_input/freeze_address.xlsx", sheet_name='Past Waterfall Freezes')
waterfall_freeze.columns = waterfall_freeze.iloc[0]
waterfall_freeze.drop([0], inplace = True)
wfreeze_last_date = str(waterfall_freeze.iloc[-1]["Date"])

freeze = pd.read_excel("excel_input/freeze_address.xlsx")
freeze.columns = freeze.iloc[0]
freeze.drop([0], inplace = True)
freeze_last_date = str(freeze.iloc[-1]["Date"])
freeze_name_date = freeze.iloc[-1]["Date"].strftime('%b %d, %Y')
freeze_date = freeze_last_date.split()[0]
isFreeze = wfreeze_last_date != freeze_last_date

In [231]:
isFreeze

False

In [232]:
# read the raw spreadsheet, downloanded from shared drive
if isFreeze:
    c_recon = pd.ExcelFile("excel_input/freeze.xlsx")
else:
    c_recon = pd.ExcelFile("excel_input/Celsius_Reconciliation_Master.xlsx")
c_recon = pd.ExcelFile("excel_input/Celsius_Reconciliation_Master.xlsx")
#c_recon = pd.ExcelFile("excel_input/freeze.xlsx")
stats = pd.read_excel(c_recon, "Coin Stats", header = None)
defi = pd.read_excel(c_recon, "DeFi Assets")
ftx = pd.read_excel(c_recon, "FTX Summary")
defi_main = pd.read_excel(c_recon, "DeFi Main")
template = pd.read_excel("excel_input/coin_apy_template.xlsx", 
                         sheet_name = "APY", 
                         header = None)
carry = pd.read_excel("excel_input/coin_apy_template.xlsx", sheet_name='Carry')
manual_adjustements = pd.read_excel("excel_input/coin_apy_template.xlsx", 
                         sheet_name = "Manual_Adjustments")

deployable = pd.read_excel("excel_input/deployable_metric.xlsx",
                           sheet_name="Undeployed & Underdeployed")
template.columns = template.iloc[0]
template = template[template["Coin"].notnull()]
template

,Coin,Bank - Balances,Celsius Network Limited (UK),Celsius Network LLC (US),Celsius Network EU UAB (LT),Celsius Network Finance,Celsius OTC,CEL Treasury,CEL Users,Loans Out,...,DD - CONVEX - gUSD,DD - CONVEX - RAI,DD - Spartan Token,DD - Banker Joe - LINK.e,DD - Alpha HomoraV2 - AVAX,DeFi-Earmarked,DD - Banker Joe - WBTC.e,DD - Aave - Matic,DD - Bancor - LINK,DD - Badger - wibBTC
0,Coin,Bank - Balances,Celsius Network Limited (UK),Celsius Network LLC (US),Celsius Network EU UAB (LT),Celsius Network Finance,Celsius OTC,CEL Treasury,CEL Users,Loans Out,...,DD - CONVEX - gUSD,DD - CONVEX - RAI,DD - Spartan Token,DD - Banker Joe - LINK.e,DD - Alpha HomoraV2 - AVAX,DeFi-Earmarked,DD - Banker Joe - WBTC.e,DD - Aave - Matic,DD - Bancor - LINK,DD - Badger - wibBTC
1,Category,undeployed,undeployed,undeployed,undeployed,undeployed,undeployed,CEL Treasury,CEL Users,Institutional Loans,...,defi,defi,defi,defi,defi,defi,defi,defi,defi,defi
2,Tier,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,...,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0
3,Default,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,wBTC (Y/N),N,N,N,NaN,N,N,N,N,N,...,N,N,N,N,N,Y,Y,N,N,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,ZEC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03567,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,ZRX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.055714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,ZUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,check,1,0.0,0.0,NaN,0.0,0.0,0.0,0.0,45,...,9,8,1,2.0,2.0,3,3,1,2.0,3


In [233]:
'''This is computing the ETHER collateral ratio for Aave and Compounds Celsius Borrows Account'''
borrow_account = defi_main[defi_main['Vault Name'] == 'Celsius Borrows Account']
col_ratio = {}
col_threshold = 2.0
protocols = ['Compound', 'Aave V2']

grouped = borrow_account[borrow_account['Coin']=='ETH'].groupby('Protocol')['sum Original Balance'].sum()
grouped = grouped/grouped.sum()

for protocol in protocols:
    temp = borrow_account[borrow_account['Protocol'] == protocol]
    borrow = sum([bal for bal in temp['sum Balance $USD'] if bal<0])
    ether = temp[temp['Coin']=='ETH']['sum Balance $USD'].values[0]
    col_ratio[protocol] = max(abs(ether/borrow) - col_threshold,0) 
    col_ratio[protocol] /= abs(ether/borrow)
    col_ratio[protocol + '_part'] = grouped[protocol]


In [234]:
deployable.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Waterfall / Recon (will get),Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,Deployable (1-4),Undeployed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Under-Deployed,NaN,NaN,NaN,NaN,NaN
1,NaN,Coin,Total Fireblocks Balances,Main Account (minimum reserve),"Treasury (margin reserves, mining, etc)",Frictional Balances 2,User's Wallets,Pending Withdrawals,Total Undeployed (Coins),Total Undeployed ($),Total Under and Undeployed,Price,NaN,Under-Deployed (Coins),Under-Deployed ($),NaN,NaN,NaN,Total Deployable
2,12020.21003,BTC,10456.41887,1310.804598,4440.137732,639.802905,6.78103,11.183738,4698.695515,204788461.9,12020.21003,43584.10994,NaN,7321.514516,319101693.5,NaN,0.0,NaN,12020.21003
3,208989.8736,ETH,213456.4275,12292.08978,34065.87579,5018.036194,11688.58831,928.625294,155409.8736,480729019.3,208989.8736,3093.297795,NaN,53580,165738895.9,NaN,NaN,NaN,208989.8736
4,740161.489,SNX,6197530.531,109076.0552,5466043.303,78603.85056,52001.59799,105.153817,570409.5743,3134543.279,740161.489,5.49525,NaN,169751.9147,932829.2286,NaN,NaN,NaN,740161.489


In [309]:
# We format the deployable tab
deployable_0 = deployable.copy()
deployable_0 = deployable.iloc[1:,[1,8]]
deployable_0.columns = deployable_0.iloc[0]
deployable_0 = deployable_0.iloc[1:]

In [236]:
# this is generating a assets/liabilities dataframe for waterfall sheet
coin_asset_liability = stats[[0,2,3]].copy()
coin_asset_liability.columns = coin_asset_liability.iloc[0]
coin_asset_liability.drop([0], inplace = True)
coin_asset_liability.dropna(axis = 0 , how = "all", inplace = True)
coin_asset_liability.reset_index(drop = True, inplace = True)
coin_asset_liability.at[0, "Coin/Asset"] = "Total"
coin_asset_liability = coin_asset_liability[coin_asset_liability["Coin/Asset"].notnull()]
coin_asset_liability.columns = ['Coin', 'Net Assets Total', 'Net Liabilities Total']
coin_asset_liability["Net Assets Total"] = coin_asset_liability["Net Assets Total"].astype("float")
coin_asset_liability["Net Liabilities Total"] = coin_asset_liability["Net Liabilities Total"].astype("float")
#coin_asset_liability.columns

In [237]:
# get the coin price from coin recon sheet, there are some coins in FTX which are not listed in the "coin stats" tab
# and have no price. I use FTX api to get their most recent price.
coin_price = stats[[0,1]]
coin_price.columns = ["Coin", "Price"]
coin_price.drop([0], inplace = True)
coin_price.dropna(axis = 0 , how = "all", inplace = True)
coin_price = coin_price[coin_price["Coin"].notnull()]
coin_price.reset_index(drop = True, inplace = True)
coin_price.loc[len(coin_price.index)] = ['Stable Coins', 1] 
srm_price = float(coin_price.loc[coin_price["Coin"] == "SRM", "Price"])
coin_price.loc[len(coin_price.index)] = ['SRM_LOCKED', srm_price] 
#coin_price


existing_coins = coin_price["Coin"].unique()
coins_need_price = ["ATLAS", "SOL","POLIS", "RAY", "SLRS", "BVOL", "YFL", "QI", "CREAM",
                   "FTT", "DASH", "THKD", "ALPHA", "WDGLD", "VSP", "ORBS", "WBTC", 
                   "3CRV", "KIN", "LRC"]
needed = []
for coin in coins_need_price:
    if coin not in existing_coins:
        needed.append(coin)
print(needed)
price_dict ={"Coin": [], "Price": []}
for coin in needed:
    url1 = (f"https://ftx.com/api/markets/{coin}/USD")
    data = requests.get(url1).json()
    price_dict["Coin"].append(coin)
    price_dict["Price"].append(data['result']["price"])
needed_price = pd.DataFrame.from_dict(price_dict)
coin_price = pd.concat([coin_price, needed_price])
needed_price

/Users/gabrielchoukroun/.conda/envs/waterfall_python/lib/python3.10/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/gabrielchoukroun/.conda/envs/waterfall_python/lib/python3.10/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


['SLRS', 'BVOL']


,Coin,Price
0,SLRS,0.33297
1,BVOL,545.45000


In [238]:
# get coin tier info 
tiers = template[template["Coin"]=="Tier"].T
tiers.reset_index(inplace = True)
tiers.columns = ["Account", "Tier"]
tiers.drop([0], inplace = True)
# there may be duplicate records in tier info
tiers.drop_duplicates(inplace = True)
# there are some rare occasions that the same account is assinged to two or more different tiers, use the first encountered
# the others are likely for some testing purposes, which are usually added to the end 
tiers = tiers[~tiers.Account.duplicated()]
filter4 = tiers['Tier'].isnull()
tiers.at[filter4, "Tier"] = "unassigned"
tiers["Tier"] = tiers["Tier"].astype("string")
#print(len(tiers))
#print(len(tiers[tiers["Tier"] == "unassigned"]))
#print(tiers.Account.nunique())
tiers["Tier"] = tiers["Tier"].apply(lambda x:x.split(".")[0])
#tiers["Tier"].unique()
#tiers

In [239]:
# mapping each accout to a category
categories = template[template["Coin"].isin(["Coin", "Category"])].T
categories.reset_index(drop = True, inplace = True)
categories.columns = ["Account", "Category"]
categories.drop([0], inplace = True)
categories.Category.fillna(value = "unassigned", inplace = True)
#print(len(categories))
#print(len(categories[categories["Category"] == "unassigned"]))
#print(categories.Account.nunique())
#categories

In [240]:
# read the cofa data
cofa_original = pd.read_excel("excel_input/coin_apy_template.xlsx", sheet_name = "COFA")
cofa_original.dropna(axis=0, how='all', inplace=True)
cofa_original.reset_index(drop = True, inplace = True)
#display(cofa_original)
cofa_original.at[cofa_original["Coin"] == "stable (USD)", "Coin"] = "Stable Coins"
cofa_melt_cols = list(cofa_original.columns)
cofa_melt_cols.remove("Coin")
#print(cofa_melt_cols)
cofa = melt(cofa_original, cofa_melt_cols, "Account", "COFA")
cofa = cofa[cofa["COFA"].notnull()]
cofa.reset_index(drop = True, inplace = True)
#cofa


In [241]:
# process the carry tab (like the cofa)
carry.dropna(axis=0, how='all', inplace=True)
carry.reset_index(drop = True, inplace = True)
#display(cofa_original)
carry.at[carry["Coin"] == "stable (USD)", "Coin"] = "Stable Coins"
carry_melt_cols = list(carry.columns)
carry_melt_cols.remove("Coin")
#print(cofa_melt_cols)
carry2 = melt(carry, carry_melt_cols, "Account", "CARRY")
carry2 = carry2[carry2["CARRY"].notnull()]
carry2.reset_index(drop = True, inplace = True)

In [242]:
# process apy data
apy = template[~template["Coin"].isin(["Coin", "Category", "Tier"])]
#apy.fillna(value = 0, inplace = True)

# people leave spaces in the excel file sometimes, need to get rid of them
apy.replace([" ", "", "  "], np.nan, inplace = True)
apy =apy[apy["Coin"] != "wBTC (Y/N)"]
#apy

/Users/gabrielchoukroun/.conda/envs/waterfall_python/lib/python3.10/site-packages/pandas/core/frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [243]:
apy2 = apy.copy()
apy2["Coin"] = apy2["Coin"] + "_APY"
apy5 = apy2.T
apy5.reset_index(inplace = True)
apy5.columns = apy5.iloc[0]
apy5.rename(columns = {"Coin": "Account"}, inplace = True)
apy5.drop([0], inplace = True)
#apy5

In [244]:
# convert dyptes to float
for col in apy5.columns:
    if col != "Account":
        apy5.loc[apy5[col].isin(template["Coin"].unique()), col] = np.nan
        apy5[col] = apy5[col].map(lambda x:np.nan if type(x)==str else x)\
            .astype(float)

In [245]:
# get the user collateral and institutional collateral data in a dataframe
# to be uploaded to waterfall sheet 
collateral = stats.copy()
collateral_p1 = collateral.iloc[:, 0].to_frame()
collateral_p1.columns = ["Coin"]
collateral_p2 = collateral.iloc[:, 4:]
collateral_p2.columns = collateral_p2.iloc[2]
collateral = pd.concat([collateral_p1, collateral_p2[["User Collateral", "Inst Collateral"]]], axis = 1)
collateral.drop([0,1,2,3], inplace = True)
collateral = collateral[collateral["Coin"].notnull()]
collateral["User Collateral"] = -1 * collateral["User Collateral"]
collateral["Inst Collateral"] = -1 * collateral["Inst Collateral"]
collateral.fillna(value = 0, inplace = True)
collateral.dtypes

Coin                object
User Collateral    float64
Inst Collateral    float64
dtype: object

In [246]:
# process stats table

# first fill the "asset" or "liability" into row 0
stats.iloc[0] = stats.iloc[0].ffill()
# drop the secondary description of asset or liability (no use)
stats.drop([1], inplace = True)

#split stats into two parts, p1 is coin name and summary, p2 is assets/liabilities
stats_p1 = stats.iloc[:, 0:3]
stats_p1.columns = stats_p1.iloc[0]
stats_p1.reset_index(drop = True, inplace = True)
stats_p1.drop([0, 1, 2], inplace = True)
stats_p1.rename(columns = {"Coin/Asset": "Coin"}, inplace = True)


stats_p2 = stats.iloc[:, 4:]

# filter according to "assets" and the detailed account name cannot be null
stats_p3 = stats_p2.loc[:, stats_p2.loc[2].notnull()]
stats_p4 = stats_p3.loc[:, stats_p3.loc[0] == "Assets"]
stats_p4.reset_index(drop = True, inplace = True)
stats_p4.columns = stats_p4.iloc[1]
stats_p4.reset_index(drop = True, inplace = True)
stats_p4.drop([0, 1, 2], inplace = True)
stats = pd.concat([stats_p1["Coin"], stats_p4], axis = 1)
stats = stats[stats["Coin"].notnull()]
stats

/Users/gabrielchoukroun/.conda/envs/waterfall_python/lib/python3.10/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/gabrielchoukroun/.conda/envs/waterfall_python/lib/python3.10/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Coin,Bank - Balances,Celsius Network Limited (UK),Celsius Network LLC (US),Celsius Network EU UAB (LT),Celsius OTC,Celsius Network Finance,CEL Treasury,CEL Users,Loans Out,...,Stakehound,Kraken Staking,Direct Staking,Mining,EAM - Balances,Others - Asset,BITFINEX,COINBASEPRO,DERIBIT,LIQUID
3,1INCH,0,0,276022.85,0,0,0,0,0,0,...,0,0,0,0,0,6666666.66,0,0,0,0
4,3CRV,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,AAVE,0,13913.27039,8112.500981,0,106.934225,5.93778,0,0,19620.65,...,0,0,0,0,0,0,0,0,0,0
6,ADA,0,1200.761924,6968903.406,1,106162.0661,268.036039,0,0,103318611,...,0,0,178989658.8,0,0,0,0,0,0,0
7,ALCX,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,YFL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
108,yveCRV-DAO,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
109,ZEC,0,212.3086,46744.36113,0,0,0,0,0,73000,...,0,0,0,0,0,0,4630.620641,0,0,0
110,ZRX,0,16561.64384,2302316.134,0,0,272.88,0,0,3500000,...,0,0,0,0,0,0,577627.223,8735.95141,0,0


In [247]:
stats[['Coin','Celsius Network LLC (US)']]

,Coin,Celsius Network LLC (US)
3,1INCH,276022.85
4,3CRV,0
5,AAVE,8112.500981
6,ADA,6968903.406
7,ALCX,0
...,...,...
107,YFL,0
108,yveCRV-DAO,0
109,ZEC,46744.36113
110,ZRX,2302316.134


In [248]:
# process DeFi assets data
defi_p1 = defi.iloc[:, 0:3]
defi_p1.drop([0, 1, 2], inplace = True)
defi_p1.rename(columns = {"Coin/Asset": "Coin"}, inplace = True)
defi_p1
defi_p2 = defi.iloc[:, 3:]

# filter according the detailed account name cannot be null
defi_p3 = defi_p2.loc[:, defi_p2.iloc[1].notnull()]
defi_p3.reset_index(drop = True, inplace = True)
defi_p3.columns = defi_p3.iloc[1]
defi_p3.reset_index(drop = True, inplace = True)
defi_p3.drop([0, 1, 2], inplace = True)
defi_p3
defi = pd.concat([defi_p1["Coin"], defi_p3], axis = 1)
defi = defi[defi["Coin"].notnull()]
#defi

In [249]:
col_ratio

{'Compound': 0.0,
 'Compound_part': 0.4027075805280056,
 'Aave V2': 0.17373601556592622,
 'Aave V2_part': 0.5593078733139167}

In [250]:
defi[defi['Coin']=='ETH']['Celsius Borrows Account']

38    551399.5719
Name: Celsius Borrows Account, dtype: object

In [251]:
col_ratio

{'Compound': 0.0,
 'Compound_part': 0.4027075805280056,
 'Aave V2': 0.17373601556592622,
 'Aave V2_part': 0.5593078733139167}

In [252]:
# ADD GAB - SPLIT ON OVERCOLLATERIZED ETH IN CELSIUS BORROWS ACCOUNT
borrow_eth = defi[defi['Coin']=='ETH']['Celsius Borrows Account'].iloc[0]
idx_eth = defi[defi['Coin']=='ETH'].index[0]
for col in ['Aave V2', 'Compound']:
    new_col = f'Overcollaterized - {col}'
    defi[new_col] = 0
    defi.at[idx_eth,new_col] = borrow_eth * col_ratio[col+'_part']* col_ratio[col]
    defi.at[idx_eth,'Celsius Borrows Account'] -= defi.at[idx_eth,new_col]

In [253]:
# process ftx summary data
ftx.dropna(axis = 0 , how = "all", inplace = True)
ftx.dropna(axis = 1 , how = "all", inplace = True)
filter_ftx = ((ftx["Total Asset"] == 0) & (ftx["Total Borrow"] == 0))
ftx = ftx[(ftx[ftx.columns[0]].notnull()) & ~filter_ftx]
ftx.drop(columns = ["Total Asset", "Total Borrow"], inplace = True)
ftx_cols = []
for col in ftx.columns:
    ftx_cols.append("FTX - " + col)
ftx.columns = ftx_cols
ftx.rename(columns = {ftx.columns[0]: "Coin"}, inplace = True)

# below lines of code will merge BTC and WBTC together
ftx_btc = ftx[ftx["Coin"].isin(["BTC", "WBTC"])]
ftx_btc = ftx_btc.append(ftx_btc.sum(numeric_only=True), ignore_index=True)
ftx_btc = ftx_btc[~ftx_btc["Coin"].isin(["BTC", "WBTC"])]
ftx_btc.reset_index(drop = True, inplace = True)
ftx_btc.loc[0, "Coin"] = "BTC"
ftx = pd.concat([ftx_btc, ftx[~ftx["Coin"].isin(["BTC", "WBTC"])]])
#ftx

In [254]:
# get update date and time
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")
#print("Updated at:", dt_string)

In [255]:
# define the stable coins, these coins are listed together under "stable coins" in waterfall sheet
stables = ['alUSD','BUSD', 'GUSD', 'LUSD', 'LUSD Curve','MCDAI', 'PAX', 'SUSD', 'TUSD', 
           'USDC', 'USDT ERC20', 'ZUSD', "USD"]

In [256]:
# create a Tier dataframe for merging purpose
tier_dict = {"Tier":["1", "2", "3", "4", "5", "unassigned"]}
coin_tiers = pd.DataFrame.from_dict(tier_dict)
coin_tiers

,Tier
0,1
1,2
2,3
3,4
4,5
5,unassigned


In [257]:
apy['DD - BANCOR - WBTC']

3      0.0
5      NaN
6      NaN
7      NaN
8      NaN
      ... 
117    NaN
118    NaN
119    NaN
121      4
123    NaN
Name: DD - BANCOR - WBTC, Length: 118, dtype: object

In [258]:
apy5[apy5['Account'] =='DD - BANCOR - WBTC']

,Account,Default_APY,1INCH_APY,3CRV_APY,AAVE_APY,ADA_APY,ALCX_APY,ALICE_APY,ALPHA_APY,alUSD_APY,...,XRP_APY,XTZ_APY,YFI_APY,YFL_APY,yveCRV-DAO_APY,ZEC_APY,ZRX_APY,ZUSD_APY,check_APY,All Coins_APY
170,DD - BANCOR - WBTC,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN


In [259]:
# merge 3 dataframes together, coin stats/Defi assets/FTX summary
coin_stats = stats.merge(defi, on = "Coin", how = "outer")
coin_stats = coin_stats.merge(ftx, on = "Coin", how = "outer")
coin_stats.fillna(value = 0, inplace = True)
coin_stats_col = []
for col in coin_stats.columns:
    coin_stats_col.append(col.strip())
coin_stats.columns = coin_stats_col
len(coin_stats.columns)
coin_stats

,Coin,Bank - Balances,Celsius Network Limited (UK),Celsius Network LLC (US),Celsius Network EU UAB (LT),Celsius OTC,Celsius Network Finance,CEL Treasury,CEL Users,Loans Out,...,FTX - DeFi,FTX - Management,FTX - Kairon,FTX - Kairon2,FTX - Brad,FTX - Jacob,FTX - cel_staking,FTX - Directional Trading 2,FTX - CEL,FTX - Borrow
0,1INCH,0.0,0.000000,2.760228e+05,0.0,0.000000,0.000000,0.0,0.0,0.000000e+00,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0000,0.0,0.000000,0.0
1,3CRV,0.0,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,0.0,0.000000e+00,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0000,0.0,0.000000,0.0
2,AAVE,0.0,13913.270390,8.112501e+03,0.0,106.934225,5.937780,0.0,0.0,1.962065e+04,...,906.741627,0.010000,0.0,0.0,0.00000,0.0,0.0000,0.0,0.000000,0.0
3,ADA,0.0,1200.761924,6.968903e+06,1.0,106162.066100,268.036039,0.0,0.0,1.033186e+08,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0000,0.0,0.000000,0.0
4,ALCX,0.0,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,0.0,0.000000e+00,...,0.411829,0.000000,0.0,0.0,0.00000,0.0,0.0000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,yveCRV-DAO,0.0,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,0.0,0.000000e+00,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0000,0.0,0.000000,0.0
106,ZEC,0.0,212.308600,4.674436e+04,0.0,0.000000,0.000000,0.0,0.0,7.300000e+04,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0000,0.0,0.000000,0.0
107,ZRX,0.0,16561.643840,2.302316e+06,0.0,0.000000,272.880000,0.0,0.0,3.500000e+06,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0000,0.0,0.000000,0.0
108,ZUSD,0.0,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,0.0,0.000000e+00,...,0.000000,0.000000,0.0,0.0,0.00000,0.0,0.0000,0.0,0.000000,0.0


In [260]:
# get a list of coins in the above dataframe and add "Stable Coins" into that list
coin_list = list(coin_stats["Coin"].unique())
coin_list.append("Stable Coins")
#coin_list

In [261]:
acc_list = np.asarray(tiers["Account"])
len(acc_list)


181

In [262]:
# this block and the following block are meant to show which accounts from coin recon sheet are not listed 
# in tier/apy sheet, or which accounts listed in the tier/apy data no longer exists in coin recon sheet
not_covered = []
for col in coin_stats.columns:
    if col not in acc_list:
        not_covered.append(col)
not_covered    

['Coin',
 'DeFi Borrows - Collateral',
 'DeFi Borrows - Tokens',
 'DeFi Assets - Assets',
 'FTX',
 'Vault Name',
 'Deployment - Vesper (OLD DO NOT USE)',
 'Matic Staking 13',
 'SGB Staking']

In [263]:
not_covered = []
for col in acc_list:
    if col not in coin_stats.columns:
        not_covered.append(col)
not_covered

['Impermanent_loss_hedge__RonSabo',
 'Hedge_Options',
 'FTX - Johannes',
 'OKEX',
 'Deployment - Vesper',
 'YD - Compound - WBTC',
 'BlockDemon ETH Staking',
 'Deployment - Stable Coin Swaps',
 'Deployment Team - COMP supply',
 'Deployment- 1INCH Staking',
 'Convex: cvxBUSD3CRV-f',
 'FTX - LONG1',
 'YD - Curve - SBTC',
 'YD - Keeper - renBTC',
 'MATIC Staking 7',
 'Deployment - 1INCH Staking (testing)',
 'YD - Curve - alUSD',
 'DD - SYNAPSIS - WETH/Arbitrum',
 'DD - SYNAPSIS - WETH/Optimism',
 'DD - SYNAPSIS - USD/Avalanche',
 'DD - SYNAPSIS - USD/Polygon',
 'DD - Benqi - Avalanche']

In [264]:
# transpose coin_stats to get it in right shape
account_toremove = ['DeFi Borrows - Assets', 'DeFi Assets - Assets', "FTX",
                   "DeFi Borrows - Collateral", "DeFi Borrows - Tokens"]
coin_stats_t = coin_stats.T
new_header = coin_stats_t.iloc[0] #grab the first row for the header
coin_stats_t = coin_stats_t[1:] #take the data less the header row
coin_stats_t.columns = new_header #set the header row as the df header
coin_stats_t.reset_index(inplace = True)
coin_stats_t.rename(columns={coin_stats_t.columns[0]: "Account" }, inplace = True)
coin_stats_t.reset_index(drop=True, inplace = True)
#coin_stats_t = coin_stats_t[coin_stats_t["Account"] != "Defi Assets"]
coin_stats_t = coin_stats_t[~coin_stats_t["Account"].isin(account_toremove)]

#coin_stats_t


In [265]:
# adding Tier info 
coin_stat_tier = tiers.merge(coin_stats_t, on = "Account", how = "right")
#coin_stat_tier["Tier"].fillna(value = "unknown", inplace = True)

coin_stat_tier['Stable Coins']= coin_stat_tier[stables].sum(axis=1)
coin_stat_tier.to_excel("excel_outputs/coin_stats_tier_live.xlsx", index = False)

#print(len(coin_stat_tier[coin_stat_tier["Tier"] == "unknown"]))
coin_stat_tier["Tier"] = coin_stat_tier["Tier"].astype("str")

In [266]:
# to calculate stable coin apy, we need to calculate for each stable coin yield = # of each stable coin * apy of each stable
# and stable coin apy = sum of all stable coin yiled / total # of stable coins
# this prepares a list of columns needed to be summed.
stable_yield = []
for coin in stables:
    stable_yield.append(coin+"_yield")
stable_yield

['alUSD_yield',
 'BUSD_yield',
 'GUSD_yield',
 'LUSD_yield',
 'LUSD Curve_yield',
 'MCDAI_yield',
 'PAX_yield',
 'SUSD_yield',
 'TUSD_yield',
 'USDC_yield',
 'USDT ERC20_yield',
 'ZUSD_yield',
 'USD_yield']

In [267]:
# ADD --------------------------
# ------------------------------
# I need to add a function that will fill _APY columns with the default values
def default_fill(row):
    if ~ np.isnan(row['Default_APY']): 
        row.loc[apy_cols] = row[apy_cols].fillna(row['Default_APY'])
    return row

In [268]:
coin_stat_tier

,Account,Tier,1INCH,3CRV,AAVE,ADA,ALCX,ALICE,ALPHA,alUSD,...,XRP,XTZ,YFI,YFL,yveCRV-DAO,ZEC,ZRX,ZUSD,SRM_LOCKED,Stable Coins
0,Bank - Balances,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.329130e+08
1,Celsius Network Limited (UK),1,0.0,0.0,13913.27039,1200.761924,0.0,0.0,0.0,0.0,...,240748.1736,1713.32935,0.1613,0.0,0.0,212.3086,16561.64384,0.0,0.0,6.727273e+05
2,Celsius Network LLC (US),1,276022.85,0.0,8112.500981,6968903.406,0.0,0.0,0.0,0.0,...,471823.5053,209033.8168,1.722868,0.0,0.0,46744.36113,2302316.134,0.0,0.0,6.117907e+07
3,Celsius Network EU UAB (LT),1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,12066878.71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.089355e+04
4,Celsius OTC,1,0.0,0.0,106.934225,106162.0661,0.0,0.0,0.0,0.0,...,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.948203e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,FTX - Jacob,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
159,FTX - cel_staking,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,909504.2707,0.000000e+00
160,FTX - Directional Trading 2,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.336017e+06
161,FTX - CEL,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6735.628024,1.519319e+06


In [269]:
# calculate the yield = # of coins * apy for each coin
coin_stat_tier_apy = coin_stat_tier.merge(apy5, on = "Account", how = "left")
#coin_stat_tier_apy.fillna(value = 0, inplace = True)

# --------- ADD GAB - Replace NA APY by the corresponding Default APY values if it exists -------------------
apy_cols = ['_APY' in col for col in coin_stat_tier_apy.columns]
coin_stat_tier_apy = coin_stat_tier_apy.apply(default_fill, axis=1, result_type='broadcast')
coin_stat_tier_apy['Stable Coins'] = coin_stat_tier_apy['Stable Coins'].astype(np.float64)
coin_stat_tier_apy = coin_stat_tier_apy.loc[:,~coin_stat_tier_apy.columns.duplicated()]
#coin_stat_tier_apy.to_excel("C:/celsius/Liquidity/coin_stat_tier_apy.xlsx", index = False)
for coin in coin_list:
    #print(coin)
    yield_name = coin + "_yield"
    coin_apy = coin + "_APY"
    if coin_apy in coin_stat_tier_apy.columns:
        coin_stat_tier_apy[yield_name] = coin_stat_tier_apy[coin] * coin_stat_tier_apy[coin_apy]
    else:
        coin_stat_tier_apy[yield_name] = 0
        coin_stat_tier_apy[coin_apy] = np.nan
coin_stat_tier_apy['Stable Coins_yield']= coin_stat_tier_apy[stable_yield].sum(axis=1)
coin_stat_tier_apy['Stable Coins_APY']= coin_stat_tier_apy['Stable Coins_yield']/ coin_stat_tier_apy['Stable Coins']
#coin_stat_tier_apy.to_excel("coin_stat_tier_apy.xlsx", index = False)

/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_26433/53041379.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  coin_stat_tier_apy[yield_name] = coin_stat_tier_apy[coin] * coin_stat_tier_apy[coin_apy]
/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_26433/53041379.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  coin_stat_tier_apy[yield_name] = 0
/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_26433/53041379.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the 

In [270]:
apy_cols = ['_APY' in col for col in coin_stat_tier_apy.columns]
coin_stat_tier_apy.loc[:,apy_cols].head()

,Default_APY,1INCH_APY,3CRV_APY,AAVE_APY,ADA_APY,ALCX_APY,ALICE_APY,ALPHA_APY,alUSD_APY,AMPL_APY,...,ZRX_APY,ZUSD_APY,check_APY,All Coins_APY,EURT_APY,KIN_APY,MSOL_APY,QRDO_APY,SGB_APY,Stable Coins_APY
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0


In [271]:
# for each coin, groupy by tier so we have a dataframe containing coin level tier data
# also calculating the apy for each coin on tier level
for coin in coin_list:
    #print(coin)
    coin_yield = coin+"_yield"
    coin_apy = coin+"_APY"
    df1 = coin_stat_tier_apy[[coin, coin_yield, "Tier"]]
    df2 = df1.groupby('Tier', as_index = False).agg({coin: "sum",
                                                    coin_yield: "sum"})
    df2[coin_apy] = df2[coin_yield]/ df2[coin]
    df2.drop(columns = [coin_yield], inplace = True)
    #display(df2)
    coin_tiers = coin_tiers.merge(df2, on = "Tier", how = "left")
coin_tiers.fillna(value = 0, inplace = True)
#coin_tiers

In [272]:
coin_tier_p1_cols = ["Tier"]
coin_tier_p2_cols = ["Tier"]
filter2 = coin_tiers["Tier"] != "unassigned"
for col in coin_tiers.columns:
    if col != "Tier" and "_APY" in col:
        coin_tier_p2_cols.append(col)
    elif col != "Tier":
        coin_tier_p1_cols.append(col)
    else:
        pass
#print(len(coin_tier_p1_cols))
#len(coin_tier_p2_cols)

In [273]:
coin_tier_p1 = coin_tiers[filter2][coin_tier_p1_cols].T
coin_tier_p1.reset_index(inplace = True)
coin_tier_p1.columns = coin_tier_p1.iloc[0]
coin_tier_p1.drop([0], inplace = True)
new_name = ["Coin"]
for col in coin_tier_p1.columns:
    if col != "Tier":
        new_name.append("Coin_Tier_"+str(col)[0])
coin_tier_p1.columns = new_name
#coin_tier_p1

In [274]:
coin_tier_p2 = coin_tiers[filter2][coin_tier_p2_cols]
coin_tier_p2.columns = coin_tier_p1_cols
coin_tier_p2 = coin_tier_p2.T
coin_tier_p2.reset_index(inplace = True)
coin_tier_p2.columns = coin_tier_p2.iloc[0]
coin_tier_p2.drop([0], inplace = True)
new_name = ["Coin"]
for col in coin_tier_p2.columns:
    if col != "Tier":
        new_name.append("APY_Tier_"+str(col)[0])
coin_tier_p2.columns = new_name
#coin_tier_p2

In [275]:
coin_tiers = coin_tier_p1.merge(coin_tier_p2, on = "Coin", how = "inner")
filter3 = ~coin_tiers["Coin"].isin(stables)
#coin_tiers.columns

## the below code is to generat the pivot_data tab

In [276]:
# Create df1 dataframe
some_dict = {"Coin":[], "Category":[], "Account": [], "Tier": [], "# of Coins": [], "APY": []}
df1 = pd.DataFrame.from_dict(some_dict)
col_names = ["Coin", "Category", "Account", "Tier", "# of Coins",
             "APY", "COFA", "USD Value", "USD Value * COFA",
             "USD Value * APY", "Deployment Status", "Manual", "CARRY",
             "USD Value * CARRY", "WBTC"]
df1

,Coin,Category,Account,Tier,# of Coins,APY


In [277]:
# Fill df1 Dataframe coin by coin
for coin in coin_list:
    if coin not in stables:
        used_cols = ["Account", "Tier", coin, coin+"_APY"]
        df2 = categories.merge(coin_stat_tier_apy[used_cols], on = "Account", how = "right")
        df2["Coin"] = coin
        df2.rename(columns = {coin: "# of Coins", coin+"_APY": "APY"}, inplace = True)
        df1 = pd.concat([df1, df2])

filter1 = df1["# of Coins"] > 10e-6
df1 = df1[filter1]
cats = list(df1["Category"].unique())
accs = list(df1["Account"].unique())
cofa_accts = list(cofa["Account"].unique())
cofa_accts.remove("Default")
carry_accts = list(carry2["Account"].unique())
carry_accts.remove("Default")
#cofa_accts

In [278]:
df1[df1['Account'] == "Celsius Network LLC (US)"][['Coin','Account',"# of Coins"]]

,Coin,Account,# of Coins
2,1INCH,Celsius Network LLC (US),276022.85
2,AAVE,Celsius Network LLC (US),8112.500981
2,ADA,Celsius Network LLC (US),6968903.406
2,AMPL,Celsius Network LLC (US),202.197954
2,ANKR,Celsius Network LLC (US),38055.138
...,...,...,...
2,XTZ,Celsius Network LLC (US),209033.8168
2,YFI,Celsius Network LLC (US),1.722868
2,ZEC,Celsius Network LLC (US),46744.36113
2,ZRX,Celsius Network LLC (US),2302316.134


In [279]:
# Add cofa
def add_cofa_carry(df1, cofa, cofa_accts, tab="COFA"):
    default_cofa = cofa[cofa["Account"] == "Default"][["Coin", tab]]
    df2 = df1.merge(default_cofa, on = "Coin", how = "left")
    for col in cofa_accts:
        #print(col)
        cofa_2 = cofa[cofa["Account"] == col]
        if col in cats:
            cofa_2.columns = ["Coin", "Category", f"{tab}-2"]
            df2 = df2.merge(cofa_2, on=["Coin", "Category"], how='left')
        elif col in accs:
            cofa_2.columns = ["Coin", "Account", f"{tab}-2"]
            df2 = df2.merge(cofa_2, on=["Coin", "Account"], how='left')
        else:
            continue
        df2_p1 = df2[df2[f"{tab}-2"].isnull()].drop(columns = [f"{tab}-2"])
        df2_p2 = df2[df2[f"{tab}-2"].notnull()]
        df2_p2[tab] = df2_p2[f"{tab}-2"]
        df2_p2.drop(columns = [f"{tab}-2"], inplace = True)
        df2 = pd.concat([df2_p1, df2_p2])
        df2.sort_values(by = ["Coin"], inplace = True)
    return df2
df2 = add_cofa_carry(df1, cofa, cofa_accts)

/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_26433/3676507738.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_p2[tab] = df2_p2[f"{tab}-2"]
/Users/gabrielchoukroun/.conda/envs/waterfall_python/lib/python3.10/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [280]:
# Add Manual Adjustements
manual_adjustements['Manual'] = True
df2['Manual'] = False
df2 = pd.concat([df2,manual_adjustements]).reset_index(drop=True)

In [281]:
deployable_0

1,Coin,Total Undeployed (Coins)
2,BTC,4698.695515
3,ETH,155409.8736
4,SNX,570409.5743
5,MATIC,13086064.3
6,DOT,45668.53691
7,BNT,8736.760869
8,CEL,611107117.3
9,LUNA,536955.3877
10,1INCH,163026.035
11,AAVE,5474.152876


In [282]:
first_accounts = ['Celsius Network Limited (UK)', 'Celsius Network LLC (US)' ]
df2[(df2['Account'].isin(first_accounts)) & (df2['Coin']=='BTC')]

,Coin,Category,Account,Tier,# of Coins,APY,COFA,Manual
140,BTC,undeployed,Celsius Network Limited (UK),1,6990.678559,0.0,0.025058,False
141,BTC,undeployed,Celsius Network LLC (US),1,3216.569517,0.0,0.025058,False


In [283]:
# Add Deployement Status
deployable_0.columns = ['Coin','Total Undeployed']
total_stable = deployable_0[deployable_0['Coin'].isin(stables)]['Total Undeployed'].sum()
deployable_0 = deployable_0.append({'Coin':'Stable Coins',
                     'Total Undeployed':total_stable}, ignore_index=True)
df2['Deployment Status'] = 'Deployed'
undeployed_accounts = df2[df2['Category'].isin(['undeployed','Undeployed'])]['Account'].unique().tolist()
# UK First than US
first_accounts = ['Celsius Network Limited (UK)', 'Celsius Network LLC (US)' ]
undeployed_accounts = [x for x in undeployed_accounts if x not in (first_accounts)]
undeployed_accounts = first_accounts + undeployed_accounts

#undeployed_accounts = np.unique(first_accounts + undeployed_accounts)
cond = df2['Account'] == 'Celsius Network LLC (US)'
df2['# of Coins'].fillna(0, inplace=True)
for account in undeployed_accounts:
    for coin in deployable_0['Coin']:
        cond1 = (df2['Account'] == account) & (df2['Coin'] == coin)
        cond2 = deployable_0['Coin'] == coin
        if sum(cond1)+sum(cond2)<2: continue

        if df2[cond1]['# of Coins'].values < deployable_0[cond2]['Total Undeployed'].values:
            deployable_0.loc[cond2, 'Total Undeployed'] -= df2.loc[cond1, '# of Coins'].values
            df2.loc[cond1, 'Deployment Status'] = 'Deployable'
        else:
            df2.loc[cond1, '# of Coins'] -= deployable_0.loc[cond2, 'Total Undeployed'].values
            df2.loc[cond1, 'Deployment Status'] = 'Liquidity Reserve'
            new_row = df2.loc[cond1]
            if  deployable_0.loc[cond2, 'Total Undeployed'].values>0:
                new_row['# of Coins'] = deployable_0.loc[cond2, 'Total Undeployed'].values
                new_row['Deployment Status'] = 'Deployable'
                df2 = df2.append(new_row, ignore_index=True)
                deployable_0.loc[cond2, 'Total Undeployed'] = 0

df2.loc[df2['Category'] == 'Underdeployed', 'Deployment Status'] = 'Deployable'

/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_26433/128963383.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row['# of Coins'] = deployable_0.loc[cond2, 'Total Undeployed'].values
/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_26433/128963383.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row['Deployment Status'] = 'Deployable'


In [284]:
#df2.to_excel("excel_outputs/cofa_2.xlsx", index = False)
# Add Price
df2 = df2.merge(coin_price, on = "Coin", how = "left")
df2["USD Value"] = df2["# of Coins"] * df2["Price"]
df2["USD Value * APY"] = df2["USD Value"] * df2["APY"]
df2["Tier"] = df2["Tier"].astype(str).apply(lambda x:x.split(".")[0])

df2["USD Value * COFA"] = df2["USD Value"] * df2["COFA"]
df2.sort_values(by = "USD Value", ascending = False, inplace = True)
df2['Category'] = df2['Category'].str.strip()
df2

,Coin,Category,Account,Tier,# of Coins,APY,COFA,Manual,Deployment Status,Price,USD Value,USD Value * APY,USD Value * COFA
148,BTC,Posted Collateral,Posted Collateral,5,50580.53,0.0,0.000000,False,Deployed,43584.10994,2204507380.343468,0.0,0.0
375,ETH,Posted Collateral,Celsius Borrows Account,5,497819.5719,0.003,0.000000,False,Deployed,3093.297795,1539904184.066114,4619712.552198,0.0
138,BTC,Institutional Loans,Loans Out,4,32017.35,0.026359,0.025058,False,Deployed,43584.10994,1395447702.387459,36782025.760076,34967326.317182
652,Stable Coins,Institutional Loans,Loans Out,4,1100972932.0,0.08975,0.101009,False,Deployed,1,1100972932.0,98812217.713877,111207933.114732
636,Stable Coins,Retail Loans,Loans,6,890363357.53,0.034,0.101009,False,Deployed,1,890363357.53,30272354.15602,89934516.856954
...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,SOL,undeployed,Celsius Network Limited (UK),1,0.0,0.0,NaN,False,Liquidity Reserve,112.275725,0.0,0.0,NaN
432,EURT,defi,DeFi-Earmarked,1,250000.0,NaN,NaN,False,Deployed,0,0.0,NaN,NaN
433,EURT,defi,DD-CONVEX-3EURpool,2,15227357.45,NaN,NaN,False,Deployed,0,0.0,NaN,NaN
434,EURT,defi,DD-CONVEX-EURT,2,5038643.976,NaN,NaN,False,Deployed,0,0.0,NaN,NaN


In [285]:
#ADD CARRY
df2 = add_cofa_carry(df2, carry2, carry_accts, tab='CARRY')
df2["USD Value * CARRY"] = df2["USD Value"] * df2["CARRY"]
df2.sort_values(by = "USD Value", ascending = False, inplace = True)

/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_26433/3676507738.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_p2[tab] = df2_p2[f"{tab}-2"]
/Users/gabrielchoukroun/.conda/envs/waterfall_python/lib/python3.10/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [286]:
df2.head(2)

,Coin,Category,Account,Tier,# of Coins,APY,COFA,Manual,Deployment Status,Price,USD Value,USD Value * APY,USD Value * COFA,CARRY,USD Value * CARRY
162,BTC,Posted Collateral,Posted Collateral,5,50580.53,0.0,0.0,False,Deployed,43584.10994,2204507380.343468,0.0,0.0,0.0,0.0
441,ETH,Posted Collateral,Celsius Borrows Account,5,497819.5719,0.003,0.0,False,Deployed,3093.297795,1539904184.066114,4619712.552198,0.0,0.0,0.0


In [287]:
# EDIT FTX JACOB Account
cond = (df2['Account'] == 'FTX - Jacob') & (df2['APY']>df2['CARRY'])
df2.loc[cond, "Category"] = "Exchange"

In [288]:
# Merge WBTC and BTC
df2['WBTC'] = 'N'
df2.loc[df2['Coin'] == 'WBTC', 'WBTC'] = 'Y'
df2['Coin'] = df2['Coin'].replace('WBTC', 'BTC')

In [289]:
df2[(df2['Account'].isin(first_accounts)) & (df2['Coin']=='BTC')]

,Coin,Category,Account,Tier,# of Coins,APY,COFA,Manual,Deployment Status,Price,USD Value,USD Value * APY,USD Value * COFA,CARRY,USD Value * CARRY,WBTC
151,BTC,undeployed,Celsius Network Limited (UK),1,4698.695515,0.0,0.025058,False,Deployable,43584.10994,204788461.900345,0.0,5131618.305015,0.020501,4198461.903087,N
153,BTC,undeployed,Celsius Network LLC (US),1,3216.569517,0.0,0.025058,False,Liquidity Reserve,43584.10994,140191319.458581,0.0,3512933.953711,0.020501,2874126.346907,N
166,BTC,undeployed,Celsius Network Limited (UK),1,2291.983044,0.0,0.025058,False,Liquidity Reserve,43584.10994,99894040.970312,0.0,2503159.037615,0.020501,2047973.413479,N
732,BTC,undeployed,Celsius Network LLC (US),1,53.568073,0.0,NaN,False,Deployable,43673.24302,2339491.452336,0.0,NaN,0.005688,13307.231356,Y


In [290]:
# process the collateral table and insert it into the liquidity tier summary
# for COFA value of collateral, use default first, there is a "Collateral" column in COFA which will override the default
cofa_collateral_p1 = cofa_original[cofa_original["Collateral"].notnull()]
cofa_collateral_p2 = cofa_original[cofa_original["Collateral"].isnull()]
cofa_collateral_p1["Default"] = cofa_collateral_p1["Collateral"]
cofa_collateral = pd.concat([cofa_collateral_p1, cofa_collateral_p2])[["Coin", "Default"]]
cofa_collateral.columns = ["Coin", "COFA"]
# Carry process
carry_collateral_p1 = carry[carry["Collateral"].notnull()]
carry_collateral_p2 = carry[carry["Collateral"].isnull()]
carry_collateral_p1["Default"] = carry_collateral_p1["Collateral"]
carry_collateral = pd.concat([carry_collateral_p1, carry_collateral_p2])[["Coin", "Default"]]
carry_collateral.columns = ["Coin", "CARRY"]

# collateral tab
collateral = collateral.merge(cofa_collateral, on = "Coin", how = "left")
collateral = collateral.merge(carry_collateral, on = "Coin", how = "left")
collateral = collateral.merge(coin_price, on = "Coin", how = "left")
collateral["User Collateral USD Value"] = collateral["User Collateral"] * collateral["Price"]
collateral["Inst Collateral USD Value"] = collateral["Inst Collateral"] * collateral["Price"]


In [291]:
# get the whole apy-template table and insert it into the liquidity tier summary
apy_template = template.drop([0])
apy_template.fillna(value = "N/A", inplace = True)
apy_template.replace([" ", "", "  "], "N/A", inplace = True)
apy_template.columns = apy_template.columns.fillna('')

In [292]:
coin_tiers[filter3].head()

,Coin,Coin_Tier_1,Coin_Tier_2,Coin_Tier_3,Coin_Tier_4,Coin_Tier_5,APY_Tier_1,APY_Tier_2,APY_Tier_3,APY_Tier_4,APY_Tier_5
0,1INCH,276022.85,0.0,0.0,0.0,6767082.1822,0.0,0.0,0.0,0.0,0.0
1,3CRV,7864563.786,80389161.786,0.0,9393661.427,5218.091403,0.0721,0.12261,0.0,0.2234,0.0
2,AAVE,22370.096863,906.745383,0.0,19621.436719,164488.820109,0.0,0.0,0.0,0.0659,0.064297
3,ADA,7076535.270063,0.0,0.0,103318611.0,0.0,0.0,0.0,0.0,0.064922,0.0
4,ALCX,0.0,0.411829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


path = "excel_outputs/Liquidity_Tier_Summary Live.xlsx"
lp = openpyxl.load_workbook(path)
lp.remove(lp['Data'])
lp.remove(lp['Price'])
lp.remove(lp['Pivot_Data'])
lp.remove(lp['APY'])
lp.remove(lp['COFA'])
lp.remove(lp['Collateral'])
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = lp
coin_tiers[filter3].to_excel(writer, sheet_name = 'Data', index = False)
coin_price.to_excel(writer, sheet_name = 'Price', index = False)
df2[col_names].to_excel(writer, sheet_name = 'Pivot_Data', index = False)
apy_template.to_excel(writer, sheet_name = 'APY', index = False)
cofa_original.to_excel(writer, sheet_name = 'COFA', index = False)
collateral.to_excel(writer, sheet_name = 'Collateral', index = False)
writer.close()
lp.close()

In [293]:
cond = (df2['Account'] == 'FTX - Jacob') & (df2['APY']>df2['CARRY'])
test = df2[col_names][cond]
test

,Coin,Category,Account,Tier,# of Coins,APY,COFA,USD Value,USD Value * COFA,USD Value * APY,Deployment Status,Manual,CARRY,USD Value * CARRY,WBTC
479,KNC,Exchange,FTX - Jacob,2,410305.9616,0.00876,0.004699,799159.511743,3755.084301,7000.637323,Deployable,False,0.004578,3658.561078,N
122,BNT,Exchange,FTX - Jacob,2,70201.52233,0.138222,0.054378,193421.119164,10517.806048,26735.02492,Deployable,False,0.056951,11015.478673,N


## following code will push the data to the shared google sheet 

In [294]:
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import datetime as dt

In [295]:
scope = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive',
]
last_updated = "Updated at - " + dt.datetime.utcnow().strftime("%m/%d/%Y, %H:%M:%S") +'UTC'
# this is ID for testing
#SPREADSHEET_ID = '1IptNC0hEhwvuyfI4m2kP9-rR-3jDAOhgQaPNkW2I_QQ'
# this is ID for waterfall sheet
SPREADSHEET_ID = "1ZkSLZH2QwHnfdSpQUWAv2qum6xzhemngjWQBJBn_KeM" #Real Last
#SPREADSHEET_ID = "1lw17fQ-oRTXPBVSDkiMGfWemtXGwzruCgsHjYBxlYoU" # LAST Test
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('write_token.json'):
        creds = Credentials.from_authorized_user_file('write_token.json', scope)
'''flow = InstalledAppFlow.from_client_secrets_file('client_secret_992941975507-pvjneopi7dhmj2mqq6arpghs6r7q31jj.json', scope)
creds = flow.run_local_server(port=0)
with open('write_token.json', 'w') as token:
    token.write(creds.to_json())'''
service = build('sheets', 'v4', credentials=creds)

In [296]:
response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Categories!A1",
    body=dict(
        majorDimension='ROWS',
        values=[[last_updated]]
)
).execute()

In [297]:
coin_tiers[filter3].head()

,Coin,Coin_Tier_1,Coin_Tier_2,Coin_Tier_3,Coin_Tier_4,Coin_Tier_5,APY_Tier_1,APY_Tier_2,APY_Tier_3,APY_Tier_4,APY_Tier_5
0,1INCH,276022.85,0.0,0.0,0.0,6767082.1822,0.0,0.0,0.0,0.0,0.0
1,3CRV,7864563.786,80389161.786,0.0,9393661.427,5218.091403,0.0721,0.12261,0.0,0.2234,0.0
2,AAVE,22370.096863,906.745383,0.0,19621.436719,164488.820109,0.0,0.0,0.0,0.0659,0.064297
3,ADA,7076535.270063,0.0,0.0,103318611.0,0.0,0.0,0.0,0.0,0.064922,0.0
4,ALCX,0.0,0.411829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [298]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Data",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Data!A1",
    body=dict(
        majorDimension='ROWS',
        values=coin_tiers[filter3].fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

In [299]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Price",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Price!A1",
    body=dict(
        majorDimension='ROWS',
        values=coin_price.fillna(value = 0).T.reset_index().T.values.tolist())
).execute()

In [300]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="APY",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="APY!A1",
    body=dict(
        majorDimension='ROWS',
        values=apy_template.fillna(value = 0).T.reset_index().T.values.tolist())
).execute()

In [301]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="COFA",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="COFA!A1",
    body=dict(
        majorDimension='ROWS',
        values=cofa_original.fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

In [302]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="CARRY",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="CARRY!A1",
    body=dict(
        majorDimension='ROWS',
        values=carry.fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

In [303]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Collateral",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Collateral!A1",
    body=dict(
        majorDimension='ROWS',
        values=collateral.fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

In [304]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Coin_Total_Asset_Liability",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Coin_Total_Asset_Liability!A1",
    body=dict(
        majorDimension='ROWS',
        values=coin_asset_liability.fillna(value = 0).T.reset_index().T.values.tolist())
).execute()

In [305]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Pivot_Data",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Pivot_Data!A1",
    body=dict(
        majorDimension='ROWS',
        values=df2[col_names].fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

## below code is for storing historical data with a date label 

# get update date and time
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")
print("Updated at:", dt_string)
date_string = dt_string[0:10]
date_string

coin_tiers["Date"] = date_string
df2["Date"] = date_string
df2_col_names = ["Date", "Coin", "Category", "Account", "Tier", "# of Coins", 
             "APY", "COFA", "Price", "USD Value", "USD Value * COFA",
             "USD Value * APY"]
coin_tiers_col_names = ['Date', 'Coin', 'Coin_Tier_1', 'Coin_Tier_2', 'Coin_Tier_3', 'Coin_Tier_4',
       'Coin_Tier_5', 'APY_Tier_1', 'APY_Tier_2', 'APY_Tier_3', 'APY_Tier_4',
       'APY_Tier_5']

# Read the liquidity data excel file and apppend new data to it
#path = "/home/fan7893/Documents/Celsius/Liquidity/Liquidity_Tier_Data.xlsx"
#path = "excel_input/Liquidity_Tier_Summary Live.xlsx"
data = coin_tiers[filter3].fillna(value = "N/A")
pivot_data = df2[col_names].fillna(value = "N/A")

# there may be some null values due to excel operation, need to filter them out
data = data[data["Date"].notnull()]
#pivot_data = pivot_data[pivot_data["Date"].notnull()]

# if running for multiple times for the same date, we need to replace the previous run data from same date
data = data[data["Date"] != date_string]
#pivot_data = pivot_data[pivot_data["Date"] != date_string]

data = pd.concat([data, coin_tiers[filter3][coin_tiers_col_names]])
pivot_data = pd.concat([pivot_data, df2[df2_col_names]])

# reinsert the updated data into the template
writer = pd.ExcelWriter(path)

# add format info
workbook = writer.book
format1 = workbook.add_format({'num_format': '$#,##0', "align": "left"})
format2 = workbook.add_format({'num_format': '#,##0.00%', "align": "left"})
format3 = workbook.add_format({'num_format': '#,##0.00', "align": "left"})
format4 = workbook.add_format({'num_format': '$#,##0.00', "align": "left"})
data.to_excel(writer, sheet_name = 'Data', index = False)
pivot_data.to_excel(writer, sheet_name = 'Pivot_Data', index = False)

# set format
worksheet = writer.sheets["Data"]
worksheet.set_column('A:B', 16)
worksheet.set_column('C:G', 18, format3)
worksheet.set_column('H:L', 18, format2)
worksheet = writer.sheets["Pivot_Data"]
worksheet.set_column('A:C', 18)
worksheet.set_column('D:D', 32)
worksheet.set_column('E:E', 16)
worksheet.set_column('F:F', 18, format3)
worksheet.set_column('G:H', 16, format2)
worksheet.set_column('I:I', 18, format4)
worksheet.set_column('J:L', 18, format1)
writer.close()

historical_data_id = "1pzlQEQE4FI4OnGrhKMfu1Skwrvs-FubYuEn-Ez5HpsM"

response = service.spreadsheets().values().clear(
    spreadsheetId=historical_data_id,
    range="Data",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=historical_data_id,
    valueInputOption='RAW',
    range="Data!A1",
    body=dict(
        majorDimension='ROWS',
        values=data.fillna(value = 0).T.reset_index().T.values.tolist())
).execute()

response = service.spreadsheets().values().clear(
    spreadsheetId=historical_data_id,
    range="Pivot_Data",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=historical_data_id,
    valueInputOption='RAW',
    range="Pivot_Data!A1",
    body=dict(
        majorDimension='ROWS',
        values=pivot_data.fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

## below code is to copy this freeze waterfall into the archive

In [306]:
if isFreeze:
    scope = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/drive.file'
    ]

    # this is ID for waterfall live, but we just updated it using freeze data
    SPREADSHEET_ID = "1ZkSLZH2QwHnfdSpQUWAv2qum6xzhemngjWQBJBn_KeM"
    #SPREADSHEET_ID = "1ULOqqkYP7DDhVkZBoEA3i-Ka4x_YBPtAPuIzb84Ekrg" # GAB LAST TEST
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('drive_file_token.json'):
            creds = Credentials.from_authorized_user_file('drive_file_token.json', scope)
    '''flow = InstalledAppFlow.from_client_secrets_file('client_secret_510599516312-aj6d72c90n3fmrbf3ou6gromil06pr8c.apps.googleusercontent.com.json', scope)
    creds = flow.run_local_server(port=0)
    with open('drive_file_token.json', 'w') as token:
        token.write(creds.to_json())'''
    service = build('drive', 'v3', credentials=creds)

    # extract the most recent freeze id from the url provided and get the file name using that id
    freeze = pd.read_excel("excel_input/freeze_address.xlsx")
    freeze.columns = freeze.iloc[0]
    freeze.drop([0], inplace = True)
    freeze_address = str(freeze.iloc[-1]["Freeze URL"])
    #print(freeze_address)
    freeze_id = freeze_address.split("/")[5]

    #get the file name of most recent freeze sheet, which include the date of the freeze
    response = service.files().get(fileId=freeze_id).execute()
    freeze_name  ="Portfolio Waterfall TEST - " + response["name"]

    # after updating the portfolio waterall live, copied it to the archive folder
    archive_folder_id = "13f2xspl16wzdHBRwhgnEkKj80zLL9Rzu"
    newfile = {'name': freeze_name,  'parents' : [archive_folder_id]}
    response = service.files().copy(fileId=SPREADSHEET_ID, body=newfile).execute()

    # get link
    link_base = "https://docs.google.com/spreadsheets/d/"
    link = link_base + response['id']
    date = freeze_last_date

In [307]:
scope = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive',
]
# this is ID for testing
SPREADSHEET_ID = "1dBf2U0nfHzDqtkrfxdm8P2hfwM_Ndb9nx4OW8HCwmcg"
if os.path.exists('write_token.json'):
        creds = Credentials.from_authorized_user_file('write_token.json', scope)
service = build('sheets', 'v4', credentials=creds)

response = service.spreadsheets().values().append(
    spreadsheetId = SPREADSHEET_ID,
    range = "Past Waterfall Freezes!A2:B4",
    body = {
        "majorDimension": "ROWS",
        "values": [[freeze_name_date,link]]
    },
    valueInputOption = 'RAW'
).execute()


"""
response = service.spreadsheets().values().batchUpdate(
    spreadsheetId=SPREADSHEET_ID,
    body={
        "valueInputOption":'RAW',
        'data':{
            'range':
            [[1,2]],
        'ValueRanges': [[date,link]]
}}).execute()"""

NameError: name 'link' is not defined

freeze_name